
# Baseline Classifier on STFT Spectrograms (Non‑TDA)

This notebook provides a **standard baseline** to compare against your homological approach.

**Pipeline**
1. Load `stft_np/` memmaps (A/B classes).
2. Extract **simple summary features** from each spectrogram (per‑frequency mean & std across time, plus first‑difference mean & std).
3. Train/test split.
4. Train a **Linear SVM** and print **test accuracy** (and F1).


In [9]:

# If an import fails, run the install cell below and re-run.
import os, json, numpy as np
from joblib import Parallel, delayed
from tqdm import tqdm
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score, classification_report

print("Imports OK")

Imports OK


In [10]:

# %pip install -q scikit-learn joblib tqdm



## Config

In [11]:

BASE_DIR     = '.'
META_PATH    = os.path.join(BASE_DIR, 'stft_np', 'meta.json')

# Limit how many clips per class (A/B) to load (adjust as you like)
N_PER_CLASS  = 1000
TEST_SIZE    = 0.2
RANDOM_STATE = 42

print("Configured.")

Configured.



## Load arrays

In [12]:

with open(META_PATH, 'r') as f:
    meta = json.load(f)

dtype_str = meta.get('dtype', 'float16')
if isinstance(dtype_str, str) and dtype_str.startswith("<class 'numpy."):
    dtype_str = dtype_str.split('.')[-1].split("'")[0]

A_path = os.path.join(BASE_DIR, 'stft_np', meta['A_memmap'])
B_path = os.path.join(BASE_DIR, 'stft_np', meta['B_memmap'])
A_shape = tuple(meta['A_shape'])
B_shape = tuple(meta['B_shape'])

A = np.memmap(A_path, dtype=dtype_str, mode='r', shape=A_shape)
B = np.memmap(B_path, dtype=dtype_str, mode='r', shape=B_shape)

print("A shape:", A.shape, "B shape:", B.shape)

A shape: (940, 1000, 257) B shape: (973, 1000, 257)



## Features: time‑summary stats per frequency bin

For each clip `X` with shape `(T=1000, F=257)` in dB ([-80, 0]):

- Convert to `[0,1]`: `Y = clip((X+80)/80)`  
- Per frequency bin (across time): compute **mean** and **std** → `2*F` dims  
- On first temporal difference `ΔY` (framewise diff): **mean** and **std** → `2*F` dims  
- Total feature length = **4×F = 1028**  


In [13]:

def spec_to_features(db_spec):
    # db_spec: (T, F) dB in [-80,0]
    Y = np.clip((db_spec.astype(np.float32) + 80.0) / 80.0, 0.0, 1.0)   # (T,F)
    mu  = Y.mean(axis=0)                                                # (F,)
    sd  = Y.std(axis=0)                                                 # (F,)
    dY  = np.diff(Y, axis=0)                                            # (T-1,F)
    dmu = dY.mean(axis=0)                                               # (F,)
    dsd = dY.std(axis=0)                                                # (F,)
    return np.concatenate([mu, sd, dmu, dsd], axis=0)                   # (4F,)

def build_dataset(A, B, n_per_class=600, seed=42, n_jobs=-1):
    rng = np.random.default_rng(seed)
    nA = min(n_per_class, A.shape[0])
    nB = min(n_per_class, B.shape[0])
    idxA = rng.choice(A.shape[0], size=nA, replace=False)
    idxB = rng.choice(B.shape[0], size=nB, replace=False)

    def one(mm, idx):
        x = np.array(mm[idx], dtype=np.float32)   # (T,F)
        return spec_to_features(x)

    XA = Parallel(n_jobs=n_jobs, prefer='threads')(
        delayed(one)(A, i) for i in tqdm(idxA, desc="A features")
    )
    XB = Parallel(n_jobs=n_jobs, prefer='threads')(
        delayed(one)(B, i) for i in tqdm(idxB, desc="B features")
    )
    X = np.vstack([XA, XB]).astype(np.float32)
    y = np.concatenate([np.zeros(nA, dtype=int), np.ones(nB, dtype=int)])
    return X, y

X, y = build_dataset(A, B, n_per_class=N_PER_CLASS, seed=RANDOM_STATE)
print("Feature matrix:", X.shape, "Labels:", y.shape)

B features: 100%|██████████| 973/973 [00:00<00:00, 1339.04it/s]


Feature matrix: (1913, 1028) Labels: (1913,)



## Train / Test split and Linear SVM

In [14]:

sss = StratifiedShuffleSplit(n_splits=1, test_size=TEST_SIZE, random_state=RANDOM_STATE)
train_idx, test_idx = next(sss.split(X, y))

X_train, X_test = X[train_idx], X[test_idx]
y_train, y_test = y[train_idx], y[test_idx]

clf = make_pipeline(
    StandardScaler(with_mean=True, with_std=True),
    LinearSVC(C=1.0, loss='hinge', random_state=RANDOM_STATE, max_iter=5000)
)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

acc = accuracy_score(y_test, y_pred)
f1  = f1_score(y_test, y_pred, average='weighted')

print(f"Baseline Test Accuracy: {acc:.4f}")
print(f"Baseline Weighted F1 : {f1:.4f}")
print()
print(classification_report(y_test, y_pred, target_names=['A','B']))

Baseline Test Accuracy: 0.7859
Baseline Weighted F1 : 0.7859

              precision    recall  f1-score   support

           A       0.78      0.79      0.78       188
           B       0.79      0.78      0.79       195

    accuracy                           0.79       383
   macro avg       0.79      0.79      0.79       383
weighted avg       0.79      0.79      0.79       383



/Users/theodorbjork/SF2956/.venv/lib/python3.9/site-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(



## (Optional) Save features for reuse

In [15]:

os.makedirs('stft_np/baseline_features', exist_ok=True)
np.save('stft_np/baseline_features/X.npy', X)
np.save('stft_np/baseline_features/y.npy', y)
print("Saved X/y to stft_np/baseline_features/")

Saved X/y to stft_np/baseline_features/
